### Author: Bogdan Bintu and Benjamin Alberts
     Date: 10/6/2022
    This is inteded to create genome-scale probes with ~250kb resolution across the human genome

In [1]:
!cd LibraryDesign3/C_Tools && python setup.py build_ext --inplace

running build_ext
copying build/lib.linux-x86_64-3.9/seqint.cpython-39-x86_64-linux-gnu.so -> 


In [2]:
import sys,os
sys.path.append(r'LibraryDesign3')
import LibraryDesigner as ld
import LibraryTools as lt
from seqint import seq2Int, seq2Int_rc
# some other packages may required
import Bio #use pip install biopython

In [3]:
#### Downloaded T2T genome from: https://www.ncbi.nlm.nih.gov/assembly/GCA_009914755.4
#### GCF_009914755.1_T2T-CHM13v2.0_genomic.fna

In [4]:
names,seqs = lt.fastaread(r'HumanGenome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna')

### Construct count tables -- only need to do it once

In [ ]:
fl = r'HumanGenome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna'
names,seqs = lt.fastaread(fl)
ct = ld.countTable(word=17,save_file=fl.replace('.fna','_17w.bin'),
                   sparse=False)

from tqdm import tqdm
for sq in tqdm(seqs):
    ct.consume(sq,verbose=False)
ct.complete(verbose=True)
ct.save()

100%|███████████████████████████████████████████| 24/24 [08:32<00:00, 21.36s/it]


Time to compute unique and clip: 430.0407826900482
Time to update matrix: 4.8050079345703125


In [ ]:
import numpy as np
arr = np.fromfile(r'HumanGenomeTranscriptome/human_genome_T2T_17w.bin',dtype=np.uint16)

seq = b"ccctaaccctaacccta"
arr[ld.seq2Int(seq)] #number of occurences in the genome of ccctaaccctaacccta

In [ ]:
fl = r'HumanTranscriptome/human__genesUnspliced.fasta'
names,seqs = lt.fastaread(fl)
ct = ld.countTable(word=17,save_file=fl.replace('.fasta','_17w.bin'),
                   sparse=False)

from tqdm import tqdm
for sq in tqdm(seqs):
    ct.consume(sq,verbose=False)
ct.complete(verbose=True)
ct.save()

### sample genome at 250kb and save 50kb centered

TO DO

Save these as fata files index by chr and location: chr5_locus_0025_50kb.fasta

In [ ]:
fl = r'HumanGenome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna'
names_gn,seqs_gn = lt.fastaread(fl)

In [ ]:
def save_fasta_loci(name_,seq_,resolution=50000,spacing=250000,save_folder = r'Probes'):
    """
    Given <name_> and <seq_> of the chromosome and resolution and spacing, 
    tile the chromome with <spacing> and save each segment of size <resolution> with a name as a fasta file.
    """
    chr_ = name_.split('chromosome')[-1].split(',')[0].replace(' ','')
    len_ = len(seq_)
    nloci = int(len_/spacing)
    for ilocus in tqdm(np.arange(nloci)):
        start = spacing*ilocus
        seq = seq_[start:(start+resolution)]#[:resolution]
        res = int(resolution/1000)
        name = 'chr'+chr_+'_locus'+str(ilocus).zfill(5)+'_'+str(res)+'kb'
        fl_name = save_folder+os.sep+name+'.fasta'
        lt.fastawrite(fl_name,[name],[seq])

In [ ]:
for i in range(len(names_gn)):
    name_,seq_ = names_gn[i],seqs_gn[i]
    save_fasta_loci(name_,seq_,resolution=50000,spacing=250000,save_folder = r'Probes')

### Run on single sequence the probe designing

In [ ]:
names[4]
save_folder = r'Probes'
ich=4
index = 25

name = 'chr5_locus'+str(index).zfill(5)+'_50kb'
seq = seqs[ich][index*250000:][:50000]
fl_name = save_folder+os.sep+name+'.fasta'
lt.fastawrite(fl_name,[name],[seq])

In [ ]:
print(fl_name)

In [5]:
in_file = r'ProbesFurnari/Probes/chr7_54773208_54823208_50kb.fasta'

save_file = in_file.replace('.fasta','.pbr')

transcriptome_fl = r'HumanTranscriptome/human__genesUnspliced_17w.bin'
genome_fl = r'HumanGenome/human_genome_T2T_17w.bin'
rep_fl = r'HumanGenome/repetitive.fasta'
top_transcriptome_fl = r'HumanTranscriptome/top_tr500_brain.fasta'
local_genome_fl = in_file


pb_designer = ld.pb_reports_class(
    sequence_dic={'file':in_file,'use_revc':True,'use_kmer':True}, #use_revc - if true  consides rc as probe
    
    map_dic={'transcriptome':{'file':transcriptome_fl,'use_revc':False,'use_kmer':True},
             'top_transcriptome':{'file':top_transcriptome_fl,'use_revc':False,'use_kmer':True},
          'genome':{'file':genome_fl,'use_revc':True,'use_kmer':True},
          'repetitive':{'file':rep_fl,'use_revc':True,'use_kmer':True},
          'local_genome':{'file':in_file,'use_revc':True,'use_kmer':True}},

    save_file=save_file,
    
    params_dic={'word_size':17,'pb_len':40,'buffer_len':0,'max_count':2**16-1,'check_on_go':True,'auto':False},
    
    dic_check={('genome','local_genome'):75,
                'transcriptome':20,
                'top_transcriptome':5,
                'repetitive':2,
               'gc':[0.25,0.75],'tm':70})

pb_designer.computeOTmaps()
pb_designer.compute_pb_report(verbose=True)
pb_designer.perform_check_end()
pb_designer.plots()
pb_designer.save_csv(name=os.path.basename(in_file).split('.')[0])

Setting attribute: map_transcriptome
Setting attribute: map_top_transcriptome
Mapping no. of seqs: 3689
Setting attribute: map_genome
Setting attribute: map_repetitive
Mapping no. of seqs: 67029
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Time(s): 365.72215509414673
Dealing with sequence: 1 out of 1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49960/49960 [00:10<00:00, 4981.67it/s]


Time(s): 17.358859062194824
Picking non-overlaping sequences.
Time(s): 0.8629136085510254


In [9]:
#### Define in_files

In [6]:
import glob
import numpy as np
in_files = glob.glob('ProbesFurnari/Probes/chr*.fasta')

In [7]:
import random
random.shuffle(in_files)

In [8]:
from tqdm.notebook import tqdm
for in_file in tqdm(in_files):
    #print(in_file)
    #modify input file
    in_file=str(in_file)
    pb_designer.sequence_dic['file'] = in_file
    #modify save file
    pb_designer.save_file = in_file.replace('.fasta','.pbr')
    if not os.path.exists(pb_designer.save_file):
        pb_designer.load_sequence_file_and_paramaters()
        #modify maps
        key='local_genome'
        pb_designer.map_dic[key]['file'] = in_file
        pb_designer.files_to_OTmap("map_"+key,pb_designer.map_dic[key])
        #compute
        pb_designer.compute_pb_report()
        pb_designer.perform_check_end()
        pb_designer.plots()
        pb_designer.save_csv(name=os.path.basename(in_file).split('.')[0])

  0%|          | 0/691 [00:00<?, ?it/s]

Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.4047105312347412
Picking non-overlaping sequences.
Time(s): 0.044121503829956055
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.420166015625
Picking non-overlaping sequences.
Time(s): 0.045081138610839844
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 3.809830904006958
Picking non-overlaping sequences.
Time(s): 0.034018516540527344
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.44348812103271484
Picking non-overlaping sequences.
Time(s): 0.049178361892700195
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.5304696559906006
Picking non-overlaping sequences.
Time(s): 0.05866718292236328
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence

Dealing with sequence: 1 out of 1
Time(s): 0.6102209091186523
Picking non-overlaping sequences.
Time(s): 0.06812572479248047
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.5451679229736328
Picking non-overlaping sequences.
Time(s): 0.05938291549682617
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 1.0612661838531494
Picking non-overlaping sequences.
Time(s): 0.12125396728515625
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 10.73057246208191
Picking non-overlaping sequences.
Time(s): 0.9188268184661865
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.7470211982727051
Picking non-overlaping sequences.
Time(s): 0.08460760116577148
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.602327823638916
Picking non-overlapin

Dealing with sequence: 1 out of 1
Time(s): 0.528362512588501
Picking non-overlaping sequences.
Time(s): 0.059302330017089844
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 4.496399879455566
Picking non-overlaping sequences.
Time(s): 0.10806012153625488
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.8872873783111572
Picking non-overlaping sequences.
Time(s): 0.1013941764831543
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.6463589668273926
Picking non-overlaping sequences.
Time(s): 0.07112622261047363
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.8235893249511719
Picking non-overlaping sequences.
Time(s): 0.0929114818572998
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 1.0630159378051758
Picking non-overlapin

Dealing with sequence: 1 out of 1
Time(s): 1.0677616596221924
Picking non-overlaping sequences.
Time(s): 0.12418532371520996
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 4.589921474456787
Picking non-overlaping sequences.
Time(s): 0.11630630493164062
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 1.618736982345581
Picking non-overlaping sequences.
Time(s): 0.19324326515197754
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.7457489967346191
Picking non-overlaping sequences.
Time(s): 0.08463501930236816
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 1.5031750202178955
Picking non-overlaping sequences.
Time(s): 0.17835283279418945
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.4972343444824219
Picking non-overlapi

...

Dealing with sequence: 1 out of 1
Time(s): 0.768651008605957
Picking non-overlaping sequences.
Time(s): 0.08568024635314941
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 1.6512494087219238
Picking non-overlaping sequences.
Time(s): 0.19826364517211914
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.7517504692077637
Picking non-overlaping sequences.
Time(s): 0.08448648452758789
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.451763391494751
Picking non-overlaping sequences.
Time(s): 0.04989933967590332
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 1.512803077697754
Picking non-overlaping sequences.
Time(s): 0.17896318435668945
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.32122135162353516
Picking non-overlapi

Dealing with sequence: 1 out of 1
Time(s): 0.9445436000823975
Picking non-overlaping sequences.
Time(s): 0.10910415649414062
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 1.6417717933654785
Picking non-overlaping sequences.
Time(s): 0.2000129222869873
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.49193358421325684
Picking non-overlaping sequences.
Time(s): 0.05464339256286621
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.28026890754699707
Picking non-overlaping sequences.
Time(s): 0.030731678009033203
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.3266732692718506
Picking non-overlaping sequences.
Time(s): 0.03561902046203613
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.553997278213501
Picking non-overl

Dealing with sequence: 1 out of 1
Time(s): 0.6342940330505371
Picking non-overlaping sequences.
Time(s): 0.07078719139099121
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.6065378189086914
Picking non-overlaping sequences.
Time(s): 0.06640839576721191
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.4004030227661133
Picking non-overlaping sequences.
Time(s): 0.043790578842163086
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.5472292900085449
Picking non-overlaping sequences.
Time(s): 0.06136298179626465
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.7765185832977295
Picking non-overlaping sequences.
Time(s): 0.09058737754821777
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.7385377883911133
Picking non-overl

Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 1.6447727680206299
Picking non-overlaping sequences.
Time(s): 0.19694232940673828
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.7634353637695312
Picking non-overlaping sequences.
Time(s): 0.08808326721191406
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.8347749710083008
Picking non-overlaping sequences.
Time(s): 0.09616637229919434
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.3950164318084717
Picking non-overlaping sequences.
Time(s): 0.043270111083984375
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.3784620761871338
Picking non-overlaping sequences.
Time(s): 0.041605234146118164
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequen

Dealing with sequence: 1 out of 1
Time(s): 1.4767699241638184
Picking non-overlaping sequences.
Time(s): 0.17507410049438477
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.4780259132385254
Picking non-overlaping sequences.
Time(s): 0.051226139068603516
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.4797646999359131
Picking non-overlaping sequences.
Time(s): 0.05290532112121582
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.6309072971343994
Picking non-overlaping sequences.
Time(s): 0.06961870193481445
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.5663139820098877
Picking non-overlaping sequences.
Time(s): 0.06172823905944824
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 0.660559892654419
Picking non-overla

In [124]:
names__,seqs__ = lt.fastaread(in_file)
map_ = ld.OTmap(seqs__,word_size=17,use_kmer=True,progress_report=False,save_file=None,sparse=False)

Mapping no. of seqs: 1


In [122]:
??ld.OTmap

In [116]:
ct = ld.countTable(word=17,save_file=None,sparse=False)
ct.consume(pb_designer.input_seqs[0],verbose=False)

In [119]:
cints,ctsints = np.unique(ct.ints,return_counts=True)
len(cints)

48708

In [39]:
np.sum([int(len(seq)/250000) for seq in seqs])

12460

In [ ]:
384 well plate - 3000$

In [40]:
12460/384*3000

97343.75

8577